In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train= pd.read_csv('/kaggle/input/leaf-classification/train.csv.zip')
df_test = pd.read_csv('/kaggle/input/leaf-classification/test.csv.zip')

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_train.head(10)

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
print(df_train.shape)
print(df_test.shape)

In [ ]:
df_train.duplicated().sum()

In [ ]:
df_train['species'].nunique()

In [ ]:
df_train.columns.values

code Reffered from other notebook for learning perpose(thanks )

## Data Preparation 

Here we use swiss army knife function to orgnize the data
<br>Also we use label encoding 

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit # we will know about that package while using it

In [ ]:
# Label Encoding
def encode(df_train,df_test):
    le = LabelEncoder().fit(df_train.species)
    labels = le.transform(df_train.species) # Species are in stings 
    
    classes = list(le.classes_) #creating list of column names for submission
    
    test_ids = df_test.id # creating variable for IDs
    
    df_train = df_train.drop(['species','id'],axis = 1) #droping columns 
    df_test = df_test.drop(['id'],axis = 1)

    return df_train, labels, classes, test_ids, df_test

df_train, labels, classes, test_ids, df_test = encode(df_train,df_test)

In [ ]:
df_train.head()

In [ ]:
df_train.shape

## Stratified TrainTest Split
### why stratified train test split?
Here are relatively large no of classes(columns) available (192 classes/columns for 990 samples/rows).This will ensure we have all classes represented in both the train and test indices.

In [ ]:
# labels is our traget columns which is we created by transforming of species colums(LabelEncoding)
labels

In [ ]:
X = df_train.values
y = labels

In [ ]:
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=5)
>>> sss.get_n_splits(X, y)

In [ ]:
sss

In [ ]:
for train_index, test_index in sss.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

## Classification
we will use 10 classification technique and printing their results. These will perform much better after tuning their Hyperparameters, this gives you a decent ballpark idea.
1. KNeighborsClassifier
2. SVC(Support Vector Classifier)
3. NuSVC(Nu-Support Vector Classification)Similar to SVC but uses a parameter to control the number of support vectors. The implementation is based on libsvm.
4. DecisionTreeClassifier
5. RandomForestClassifier
6. AdaBoostClassifier
7. GradientBoostingClassifier
8. GaussianNB
9. LinearDiscriminantAnalysis
10. QuadraticDiscriminantAnalysis

In [ ]:
# importing libraries

from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC,LinearSVC,NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis
from sklearn.metrics import log_loss

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel='rbf',C = 0.025, probability= True),
    NuSVC(probability = True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()
]

Creating interation for these 10 algorithm and printing result then we select higher algorithm which having high accuracy and less log loss

In [ ]:
# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"] # created list 
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train) # fit method for applying algorithm 
    name = clf.__class__.__name__
    
    print("="*30) ### printing = in 30 times
    print(name) ### printing name of algorithm
    
    print('****Results****') 
    train_predictions = clf.predict(X_test)   ### predicting y_predict
    acc = accuracy_score(y_test, train_predictions) # now compairing y predict(train_prediction) with y_test
    print("Accuracy: {:.4%}".format(acc)) ### printing accuracy  with 4 decimal with pecentage mark 
    
    train_predictions = clf.predict_proba(X_test) ### Probability estimates y_predict_probability(train_predictions)
    ll = log_loss(y_test, train_predictions) ### appllying log loss on y_test and y_predict_probability(train_predictions)
    print("Log Loss: {}".format(ll)) ### printing log loss  
    
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols) ## creating new dataframe with 
                                                                        ## name of log_entry considering name ,
                                                                          ## accuracy score(this score not in pecentage form hence it multiply by 100)
                                                                            ##logloss stored each iteratin value in log_cols
    log = log.append(log_entry) ### append those entry in log 
    
print("="*30)
    

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_color_codes("muted")
sns.barplot(y='Classifier', x='Accuracy', data=log, color="b")

plt.xlabel('Accuracy %')
plt.title('Classifier Accuracy')
plt.show()

sns.set_color_codes("muted")
sns.barplot(x='Log Loss', y='Classifier', data=log, color="g")

plt.xlabel('Log Loss')
plt.title('Classifier Log Loss')
plt.show()

In [ ]:
log.head(10)

Random Forest Classifier has the highest accuracy  also it has less log loss so we choosing Random forest Classifier as our model 

In [ ]:
# now predicting test values 
chosen_clf = RandomForestClassifier()
chosen_clf.fit(X_train,y_train)
test_predictions = chosen_clf.predict_proba(df_test)

# Format DataFrame
submission = pd.DataFrame(test_predictions, columns=classes)
submission.insert(0, 'id', test_ids)
submission.reset_index()

# saving submission
submission.to_csv('submission.csv', index = False)
submission.tail()